In [11]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from 
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

# PARAMETERS
#-----------
degree = 3
fraction_used = 1

Directory has been set before:
/home/harry/Documents/customer-loyalty/


In [21]:
# Loading files for training and testing

# Load plain train data and test data:
data = pd.read_csv('all/train.csv')
testdata = pd.read_csv('all/test.csv')

# --------------------------------------------------------------------------------------------------------------------------

# Load all features in the folder called 'features': (comment these lines out for custom set of features)
# for f in listdir(current_dir + "/features"):
#     if (f[:4] == "test"):
#         print("this is a test feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
#     if (f[:5] == "train"):
#         print("this is a train feature : " + f)
#         temp = pd.read_csv(current_dir + "/features/" + f)
#         data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )

# --------------------------------------------------------------------------------------------------------------------------

# Load a custom set of features: (comment these lines out for all features)
data = data.drop(['feature_1'],axis=1)
testdata = testdata.drop(['feature_1'],axis=1)
data = data.drop(['feature_2'],axis=1)
testdata = testdata.drop(['feature_2'],axis=1)
feature_names = ['test_feature_fam.csv','train_feature_fam.csv','test_feature_feature_1_is_1.csv', \
                
                 'test_feature_feature_1_is_2.csv','test_feature_feature_1_is_3.csv','test_feature_feature_1_is_4.csv', \
                 'test_feature_feature_1_is_5.csv','train_feature_feature_1_is_1.csv','train_feature_feature_1_is_2.csv', \
                 'train_feature_feature_1_is_3.csv','train_feature_feature_1_is_4.csv','train_feature_feature_1_is_5.csv', \
                 'test_feature_avg_purchases.csv','train_feature_avg_purchases.csv','test_feature_feature_2_is_1.csv', \
                 'test_feature_feature_2_is_2.csv','test_feature_feature_2_is_3.csv','train_feature_feature_2_is_1.csv', \
                 'train_feature_feature_2_is_2.csv','train_feature_feature_2_is_3.csv']

for f in feature_names:
    if (f[:4] == "test"):
        print("this is a test feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        testdata.insert(testdata.shape[1], f[13:-4], temp[f[13:-4]] )
    if (f[:5] == "train"):
        print("this is a train feature : " + f)
        temp = pd.read_csv(current_dir + "/features/" + f)
        data.insert(data.shape[1]-1, f[14:-4], temp[f[14:-4]] )

this is a test feature : test_feature_fam.csv
this is a train feature : train_feature_fam.csv
this is a test feature : test_feature_feature_1_is_1.csv
this is a test feature : test_feature_feature_1_is_2.csv
this is a test feature : test_feature_feature_1_is_3.csv
this is a test feature : test_feature_feature_1_is_4.csv
this is a test feature : test_feature_feature_1_is_5.csv
this is a train feature : train_feature_feature_1_is_1.csv
this is a train feature : train_feature_feature_1_is_2.csv
this is a train feature : train_feature_feature_1_is_3.csv
this is a train feature : train_feature_feature_1_is_4.csv
this is a train feature : train_feature_feature_1_is_5.csv
this is a test feature : test_feature_avg_purchases.csv
this is a train feature : train_feature_avg_purchases.csv
this is a test feature : test_feature_feature_2_is_1.csv
this is a test feature : test_feature_feature_2_is_2.csv
this is a test feature : test_feature_feature_2_is_3.csv
this is a train feature : train_feature_f

## Training
Training will be done in the following cells. First a random part of the training data is assigned as training set and the leftovers will be the validation set. By making the _trainsize_ variable round(train.shape[0]/1), the entirety of the training data is used as training set, so no evaluation will be done in the section _Testing within training set_.

In [13]:
train = np.array(data)
np.random.shuffle(train)
fraction_used = fraction_used     # Fraction of dataset used for training (1 = entire dataset)
trainsize = round(train.shape[0] * fraction_used)
features = train.shape[1] - 3    # Amount of features in both training and test set

# A part of the training set can be used as a test set,
# so the Root Mean Squared Error can be found
train_l = train[:trainsize,features+2]
train_x = train[:trainsize,2:features+2]

# A validation sets if Root Mean Squared Error needs to be calculated without making a submission to Kaggle
val_l = train[trainsize:,features+2]
val_x = train[trainsize:,2:features+2]

poly = PolynomialFeatures(degree=degree)
train_x = poly.fit_transform(train_x)
try:
    val_x = poly.fit_transform(val_x)
except:
    pass

In [14]:
# Different regression models
model_LR = LinearRegression().fit(train_x,train_l)

# SVM took very long and didn't give better results
# model_SVM = svm.SVR()
# model_SVM.fit(train_x,train_l)

model = model_LR # Selected model

## Testing within training set

In [15]:
def RMSerror(y,y_pred):
    if y.shape[0] != y_pred.shape[0]:
        print("Root Mean Squared Error can't be calculated if number" \
              " of predictions isn't equal to number of labels!")
        return
    
    SE = 0
    for i in range(y.shape[0]):
        SE += np.square(y[i] - y_pred[i])
    
    RMSE = np.sqrt(SE / y.shape[0])
    return RMSE

In [16]:
if val_x.shape[0] > 0:
    predictions = model.predict(val_x)
    print(RMSerror(val_l,predictions))
else:
    print("No testing can be performed within the training set" \
         ", because the entire training set is used for the model")

No testing can be performed within the training set, because the entire training set is used for the model


## Predictions for test set (creating a submission .csv file)
Make sure to use the entire training set in the the "Training" section

In [17]:
testset = np.array(testdata)
testset_x = testset[:,2:features+2]

poly = PolynomialFeatures(degree=degree)
testset_x = poly.fit_transform(testset_x)

In [18]:
submission_predictions = model.predict(testset_x)

submission.target = submission_predictions

In [19]:
current_time = datetime.datetime.now().strftime("_%Y-%m-%d_%H-%M-%S")
save_name = 'submission_v' + str(len(listdir(current_dir + '/submissions')) + 1) + current_time + '.csv'
submission.to_csv('submissions/' + save_name, index = False)
print('Submission was saved under the name:' + '\033[1m' + save_name + '\033[0;0m')

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

Submission was saved under the name:submission_v10_2019-01-21_14-47-28.csv


In [20]:
# run this after previous cel to see the submission
submission

,card_id,target
0,C_ID_0ab67a22ab,-0.409531
1,C_ID_130fd0cbdd,-0.497557
2,C_ID_b709037bc5,-0.210308
3,C_ID_d27d835a9f,0.186131
4,C_ID_2b5e3df5c2,-0.735127
5,C_ID_5814b4f13c,-0.272642
6,C_ID_a1b3c75277,0.001597
7,C_ID_f7cada36d3,-0.514948
8,C_ID_9d2bc8dfc4,-0.254103
9,C_ID_6d8dba8475,-0.730203
